In [3]:
!conda install pytorch
!./run_unsupervised_nmt.sh

/bin/sh: 1: conda: not found

===== Input parameters =====
 TARGET       : 
 MONO_DATASET : en:./data/mono/mono.en.tok.60000.pth,,;:./data/mono/mono..tok.60000.pth,,
 PARA_DATASET : en-:,./data/para/en.dev.en.60000.pth,./data/para/en.dev..60000.pth
 PRETRAINED   : ./data/mono/all.en-.60000.vec
 
Traceback (most recent call last):
  File "main.py", line 12, in <module>
    from src.data.loader import check_all_data_params, load_data
  File "/home/jade/UnsupervisedMT/NMT/src/data/loader.py", line 10, in <module>
    import torch
ImportError: No module named 'torch'
